In [ ]:
import pandas as pd
import time
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats

# Define seasons of interest (ending years)
target_seasons = list(range(2000, 2025))
nba_teams = teams.get_teams()
data_rows = []

print("📊 Pulling NBA team stats 2000–2024...")

for team in nba_teams:
    team_id = team['id']
    team_name = team['full_name']

    try:
        stats = teamyearbyyearstats.TeamYearByYearStats(team_id=team_id)
        df = stats.get_data_frames()[0]

        # Parse season end year from "YEAR" column (like "2000-01")
        df['Season'] = df['YEAR'].str[-2:].astype(int) + 2000
        # Special case for 1999-00 (which ends in 2000)
        df.loc[df['YEAR'].str.startswith('1999'), 'Season'] = 2000

        # Filter for our target seasons
        df = df[df['Season'].isin(target_seasons)]

        for _, row in df.iterrows():
            gp = row['GP']
            fg3a = row['FG3A']
            fg3m = row['FG3M']

            data_rows.append({
                'TeamName': team_name,
                'Season': row['Season'],
                'GP': gp,
                'WINS': row['WINS'],
                'LOSSES': row['LOSSES'],
                'WinPCT': row['WIN_PCT'],
                'FG3A_per_game': fg3a,
                'FG3M_per_game': fg3m,
                'FG3_PCT': row['FG3_PCT'],
                'FG3A_total': round(fg3a * gp),
                'FG3M_total': round(fg3m * gp)
            })

        print(f"✓ {team_name}")
        time.sleep(0.8)

    except Exception as e:
        print(f"✗ Failed for {team_name}: {e}")

# Save results
df_final = pd.DataFrame(data_rows)
if not df_final.empty:
    df_final.to_csv("nba_3pt_win_data_2000_2024.csv", index=False)
    print("\n✅ Final dataset saved: nba_3pt_win_data_2000_2024.csv")
else:
    print("❌ Still no data — something deeper may be blocking access (e.g., network/firewall/API changes).")


📊 Pulling NBA team stats 2000–2024...
✓ Atlanta Hawks
✓ Boston Celtics
✓ Cleveland Cavaliers
✓ New Orleans Pelicans
✓ Chicago Bulls
✓ Dallas Mavericks
✓ Denver Nuggets
✓ Golden State Warriors
✓ Houston Rockets
✓ Los Angeles Clippers
✓ Los Angeles Lakers
✓ Miami Heat
✓ Milwaukee Bucks
✓ Minnesota Timberwolves
✓ Brooklyn Nets
✓ New York Knicks
✓ Orlando Magic
✓ Indiana Pacers
✓ Philadelphia 76ers
✓ Phoenix Suns
✓ Portland Trail Blazers
✓ Sacramento Kings
✓ San Antonio Spurs
✓ Oklahoma City Thunder
✓ Toronto Raptors
✓ Utah Jazz
✓ Memphis Grizzlies
✓ Washington Wizards
✓ Detroit Pistons
✓ Charlotte Hornets

✅ Final dataset saved: nba_3pt_win_data_2000_2024.csv
